In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm_notebook as tqdm

In [2]:
path = '.\\valid\\'

In [3]:
path2 = '.\\res\\'

Считываем данные по трекам с привязанными остановками

## НЕ ЗАБЫТЬ ПОМЕНЯТЬ ИМЯ ФАЙЛА

In [4]:
res_stop = pd.read_csv(path2+'res_TB_2019-10-09.csv')

In [5]:
res_stop=res_stop.rename(columns={'MODULEID':'tabelnum'})

Определяем времена начала и конца для каждого табельного номера

In [6]:
mr_id_marsh_valid_id = pd.read_csv('./data/tol_marsh_TB.csv', sep = ';', encoding = 'windows-1251')

In [7]:
mr_id_marsh_valid_id.head()

,Код Маршрута,Код предприятия,Название маршрута,Вид транспорта,mr_id,mr_name
0,90103456789,26,1 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,3473581925,Маршрут 1
1,42403456789,26,13 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227623020,Маршрут 13
2,42503456789,26,14 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227623040,Маршрут 14
3,42603456789,26,18 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2483694506,Маршрут 18
4,42103456789,26,2 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227623067,Маршрут 2


In [8]:
validation= pd.read_csv(path+'2019.10.09.csv', sep = ',', encoding = 'cp1251')
validation=validation.merge(mr_id_marsh_valid_id,on =['Код Маршрута'])
validation=validation[validation['mr_id'].isin(res_stop['mr_id'].unique())]
validation['time']=pd.to_datetime(validation['Дата и время поездки'])
validation['hour']=validation['time'].dt.hour
validation.loc[validation.hour ==0, 'hour'] = 24
validation.loc[validation.hour ==1, 'hour'] = 25
validation.loc[validation.hour ==2, 'hour'] = 26
validation.loc[validation.hour ==3, 'hour'] = 27
validation['min']=validation['time'].dt.minute
validation['sec']=validation['time'].dt.second
validation['time']=(validation['hour'].astype(int))*3600+(validation['min'].astype(int))*60+(validation['sec'].astype(int))
validation=validation.drop(['hour'], axis=1).drop(['min'], axis=1).drop(['sec'], axis=1)
validation_start=pd.DataFrame(validation.groupby(['Код Маршрута','Табельный номер']).agg({'time': np.min})).reset_index().rename(columns={"time": "start"})
validation_finish=pd.DataFrame(validation.groupby(['Код Маршрута','Табельный номер']).agg({'time': np.max})).reset_index().rename(columns={"time": "finish"})
validation_start_finish=validation_start.merge(validation_finish, on=['Код Маршрута','Табельный номер'])
validation2=validation.merge(validation_start_finish, on=['Код Маршрута','Табельный номер'])
del validation_start
del validation_finish
del validation

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## НЕ ЗАБЫТЬ ПОМЕНЯТЬ ИМЯ ФАЙЛА

In [9]:
df_4 = pd.read_csv('./data/TB_2019-10-09.csv')
df_4=df_4.rename(columns={'MODULEID':'tabelnum'})
df_4['time2']=pd.to_datetime(df_4['time'])
df_4['hour']=df_4['time2'].dt.hour
df_4['min']=df_4['time2'].dt.minute
df_4['sec']=df_4['time2'].dt.second
df_4.loc[df_4.hour ==0, 'hour'] = 24
df_4.loc[df_4.hour ==1, 'hour'] = 25
df_4.loc[df_4.hour ==2, 'hour'] = 26
df_4.loc[df_4.hour ==3, 'hour'] = 27
df_4['time2']=(df_4['hour'].astype(int))*3600+(df_4['min'].astype(int))*60+(df_4['sec'].astype(int))
df_4=df_4.drop(['hour'], axis=1).drop(['min'], axis=1).drop(['sec'], axis=1)
df_4_start=pd.DataFrame(df_4.groupby(['mr_id','tabelnum']).agg({'time2': np.min})).reset_index().rename(columns={"time2": "start"})
df_4_finish=pd.DataFrame(df_4.groupby(['mr_id','tabelnum']).agg({'time2': np.max})).reset_index().rename(columns={"time2": "finish"})
df_4_start_finish=df_4_start.merge(df_4_finish, on=['mr_id','tabelnum']).sort_values(by=['mr_id', 'start'])
del df_4_start
del df_4_finish

In [10]:
validation_start_finish2=validation_start_finish.merge(mr_id_marsh_valid_id,on =['Код Маршрута'])
mrid=validation_start_finish2['mr_id'].unique()
df_4_start_finish=df_4_start_finish[df_4_start_finish['mr_id'].isin(mrid)]
validation_start_finish2=validation_start_finish2.sort_values(by=['mr_id', 'start'])
df_4_start_finish=df_4_start_finish.sort_values(by=['mr_id', 'start'])

In [11]:
validation_start_finish2.head()

,Код Маршрута,Табельный номер,start,finish,Код предприятия,Название маршрута,Вид транспорта,mr_id,mr_name
27,42303456789,6091,17523,48246,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7
25,42303456789,5377,18926,50358,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7
23,42303456789,3970,21680,64787,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7
21,42303456789,2620,21823,63107,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7
32,42303456789,6879,22086,50081,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7


In [12]:
df_4_start_finish_mem=df_4_start_finish

In [13]:
res=validation_start_finish2

1. Раскидываем машины по табельным номерам валидаций. Жесткое условие (все валидации внутри трека)

In [14]:
k=0
res=validation_start_finish2
tabelnum = []
start = []
finish = []
tracks=pd.DataFrame()
for index, row in res.iterrows():
    stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['start']<=row['start'])
                    &(df_4_start_finish['finish']>row['finish'])]
    df_4_start_finish=df_4_start_finish[~df_4_start_finish.isin(stops[0:1])].dropna(how = 'all')
    if stops.empty:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)
    else:
        tabelnum_unic=stops['tabelnum'].values
        tabelnum.append(tabelnum_unic[0])
        start_unic=stops['start'].values
        start.append(start_unic[0])
        finish_unic=stops['finish'].values
        finish.append(finish_unic[0])
res['tabelnum']=tabelnum
res['start_track']=start
res['finish_track']=finish

In [15]:
res.head()

,Код Маршрута,Табельный номер,start,finish,Код предприятия,Название маршрута,Вид транспорта,mr_id,mr_name,tabelnum,start_track,finish_track
27,42303456789,6091,17523,48246,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,7356.0,14404.0,100797.0
25,42303456789,5377,18926,50358,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,3847.0,14407.0,100795.0
23,42303456789,3970,21680,64787,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,7378.0,14411.0,100758.0
21,42303456789,2620,21823,63107,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,7502.0,14412.0,100785.0
32,42303456789,6879,22086,50081,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,7388.0,14415.0,100793.0


In [16]:
len(df_4_start_finish)

12

2. Раскидываем машины по табельным номерам валидаций. Мягкое условие 1 (все валидации после начала трека)

In [17]:
k=0
tabelnum = []
start = []
finish = []
stops=pd.DataFrame()
for index, row in res.iterrows():
    if row['tabelnum']==0:
        stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['start']<=row['start'])]
        df_4_start_finish=df_4_start_finish[~df_4_start_finish.isin(stops[0:1])].dropna(how = 'all')
        if stops.empty:
            tabelnum.append(0)
            start.append(0)
            finish.append(0)
        else:
            tabelnum_unic=stops['tabelnum'].values
            tabelnum.append(tabelnum_unic[0])
            start_unic=stops['start'].values
            start.append(start_unic[0])
            finish_unic=stops['finish'].values
            finish.append(finish_unic[0])
    else:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)        
res['tabelnum_2']=tabelnum
res['start_track_2']=start
res['finish_track_2']=finish

3. Раскидываем машины по табельным номерам валидаций. Мягкое условие 2 (все валидации до конца трека)

In [18]:
k=0
tabelnum = []
start = []
finish = []
stops=pd.DataFrame()
for index, row in res.iterrows():
    if row['tabelnum']==0:
        stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['finish']>row['finish'])]
        df_4_start_finish=df_4_start_finish[~df_4_start_finish.isin(stops[0:1])].dropna(how = 'all')
        if stops.empty:
            tabelnum.append(0)
            start.append(0)
            finish.append(0)
        else:
            tabelnum_unic=stops['tabelnum'].values
            tabelnum.append(tabelnum_unic[0])
            start_unic=stops['start'].values
            start.append(start_unic[0])
            finish_unic=stops['finish'].values
            finish.append(finish_unic[0])
    else:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)        
res['tabelnum_3']=tabelnum
res['start_track_3']=start
res['finish_track_3']=finish

In [19]:
res.head()

,Код Маршрута,Табельный номер,start,finish,Код предприятия,Название маршрута,Вид транспорта,mr_id,mr_name,tabelnum,start_track,finish_track,tabelnum_2,start_track_2,finish_track_2,tabelnum_3,start_track_3,finish_track_3
27,42303456789,6091,17523,48246,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,7356.0,14404.0,100797.0,0.0,0.0,0.0,0.0,0.0,0.0
25,42303456789,5377,18926,50358,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,3847.0,14407.0,100795.0,0.0,0.0,0.0,0.0,0.0,0.0
23,42303456789,3970,21680,64787,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,7378.0,14411.0,100758.0,0.0,0.0,0.0,0.0,0.0,0.0
21,42303456789,2620,21823,63107,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,7502.0,14412.0,100785.0,0.0,0.0,0.0,0.0,0.0,0.0
32,42303456789,6879,22086,50081,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,7388.0,14415.0,100793.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
len(df_4_start_finish)

1

4.Дозаполняем табельные номера тем, что не вошло

In [21]:
k=0
df_4_start_finish=df_4_start_finish_mem
tabelnum = []
start = []
finish = []
stops=pd.DataFrame()
for index, row in res.iterrows():
    if row['tabelnum']==0:
        stops=df_4_start_finish[(df_4_start_finish['mr_id']==row['mr_id'])
                    &(df_4_start_finish['finish']>row['finish'])]
        if stops.empty:
            tabelnum.append(0)
            start.append(0)
            finish.append(0)
        else:
            tabelnum_unic=stops['tabelnum'].values
            tabelnum.append(tabelnum_unic[0])
            start_unic=stops['start'].values
            start.append(start_unic[0])
            finish_unic=stops['finish'].values
            finish.append(finish_unic[0])
    else:
        tabelnum.append(0)
        start.append(0)
        finish.append(0)        
res['tabelnum_4']=tabelnum
res['start_track_4']=start
res['finish_track_4']=finish

In [22]:
res.to_csv(path+'for_mege_tabelnum_2019-10-09.csv')
#df_4_start_finish.to_csv(path+'for_mege_tabelnum_ost_29.csv')

In [23]:
len(res[res['tabelnum_3']>0])

3

In [24]:
len(res[res['tabelnum']==0])

23

Перезаписываем данные в столбцах

In [25]:
res.loc[res.tabelnum==0, 'tabelnum'] = res.loc[res.tabelnum==0, 'tabelnum_3']
res.loc[res.tabelnum==0, 'tabelnum'] = res.loc[res.tabelnum==0, 'tabelnum_2']
res.loc[res.start_track==0, 'start_track'] = res.loc[res.start_track==0, 'start_track_2']
res.loc[res.start_track==0, 'start_track'] = res.loc[res.start_track==0, 'start_track_3']
res.loc[res.finish_track==0, 'finish_track'] = res.loc[res.finish_track==0, 'finish_track_2']
res.loc[res.finish_track==0, 'finish_track'] = res.loc[res.finish_track==0, 'finish_track_3']

In [26]:
res.loc[res.tabelnum==0, 'tabelnum'] = res.loc[res.tabelnum==0, 'tabelnum_4']
res.loc[res.start_track==0, 'start_track'] = res.loc[res.start_track==0, 'start_track_4']
res.loc[res.finish_track==0, 'finish_track'] = res.loc[res.finish_track==0, 'finish_track_4']

Пересохранили табельный номер как int, чтобы правильнее можно было merge делать

In [27]:
table_num_merge_29=res[['Табельный номер','mr_id','tabelnum']]

In [28]:
len(validation2['mr_id'].unique())

7

In [29]:
table_num_merge_29[['Табельный номер','tabelnum']]

,Табельный номер,tabelnum
27,6091,7356.0
25,5377,3847.0
23,3970,7378.0
21,2620,7502.0
32,6879,7388.0
...,...,...
15,3946,3849.0
12,1102,7706.0
13,1198,7698.0
65,6453,7394.0


In [30]:
table_num_merge_29.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 27 to 64
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Табельный номер  66 non-null     object 
 1   mr_id            66 non-null     int64  
 2   tabelnum         66 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.1+ KB


In [31]:
validation4=validation2.merge(table_num_merge_29,on =['mr_id','Табельный номер'],how='left')
validation4=validation4.sort_values(by=['mr_id','Табельный номер','time'])

In [32]:
validation4

,Дата и время поездки,Терминал,Номер билета,Номер кристалла,TCAT,NUM_TRIP,Код Маршрута,Табельный номер,Код предприятия_x,Код предприятия_y,Название маршрута,Вид транспорта,mr_id,mr_name,time,start,finish,tabelnum
3332,2019-10-09 06:16:53,82784787,1,40AB3B79,БЕТК ТЛТ,0,42303456789,2500,26,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,22613,22613,64228,7366.0
3333,2019-10-09 06:16:56,82784787,2,8E756685,СОЦ,73,42303456789,2500,26,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,22616,22613,64228,7366.0
3334,2019-10-09 06:19:35,82784787,3,ADD3F69B,СОЦ,74,42303456789,2500,26,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,22775,22613,64228,7366.0
3335,2019-10-09 06:21:17,82784787,6,B30C5842,СОЦ,82,42303456789,2500,26,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,22877,22613,64228,7366.0
3514,2019-10-09 06:21:30,82784787,7,NaN,БАНК,0,42303456789,2500,26,26,7 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2227622944,Маршрут 7,22890,22613,64228,7366.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9460,2019-10-09 08:08:01,82784789,68,00219966,БЕТК ТЛТ,0,42803456789,6453,26,26,57 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2488443866,Маршрут 57,29281,21223,30036,7394.0
9461,2019-10-09 08:08:43,82784789,70,8312404B,СОЦ,72,42803456789,6453,26,26,57 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2488443866,Маршрут 57,29323,21223,30036,7394.0
9462,2019-10-09 08:10:04,82784789,72,A375304B,СОЦ,74,42803456789,6453,26,26,57 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2488443866,Маршрут 57,29404,21223,30036,7394.0
9463,2019-10-09 08:13:26,82784789,74,C5FE6BF9,СОЦ,80,42803456789,6453,26,26,57 ГОРОДСКОЙ МАРШРУТ,ТРОЛЛЕЙБУС,2488443866,Маршрут 57,29606,21223,30036,7394.0


In [33]:
del validation2

Переводим столбец с временем в формат INT и прописываем для каждого табельного номера время начала и конца

In [34]:
res_stop

,mr_id,rl_id,tabelnum,rc_orderby,len,len.1,ssId,time,time_start,time_finish,ss_lat,ss_long,geometry
0,2.227623e+09,2.278607e+09,2958,2.0,85.140601,85.140601,stop__9854705,39633,39633,39691.0,53.477451,49.474193,POINT (49.474156 53.478216)
1,2.227623e+09,2.278607e+09,2958,3.0,13.259377,13.259377,stop__9854703,39691,39691,39773.0,53.480778,49.474305,POINT (49.474192 53.480876)
2,2.227623e+09,2.278607e+09,2958,4.0,48.488333,48.488333,stop__9854681,39773,39773,39894.0,53.484178,49.474376,POINT (49.474308 53.484612)
3,2.227623e+09,2.278607e+09,2958,5.0,16.785850,16.785850,stop__9854335,39894,39894,39934.0,53.487297,49.469209,POINT (49.46898 53.48723199999999)
4,2.227623e+09,2.278607e+09,2958,6.0,21.439911,21.439911,stop__9854337,39934,39934,40208.0,53.487696,49.465350,POINT (49.465052 53.48761999999999)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13744,2.488444e+09,2.488965e+09,7394,14.0,20.620268,20.620268,stop__9854566,44653,44653,45174.0,53.512005,49.450001,POINT (49.44983 53.51216)
13745,2.488444e+09,2.488965e+09,7394,16.0,32.346212,32.346212,stop__9854338,45174,45174,45226.0,53.487553,49.464127,POINT (49.464333 53.48781700000001)
13746,2.488444e+09,2.488965e+09,7394,17.0,79.860440,79.860440,stop__9854336,45226,45226,45314.0,53.486990,49.469868,POINT (49.468792 53.487315)
13747,2.488444e+09,2.488965e+09,7394,18.0,43.150032,43.150032,stop__9854682,45314,45314,45398.0,53.485638,49.473772,POINT (49.474105 53.485972)


In [35]:
validation4['tr_id']='';

In [36]:
validation4=validation4[validation4['mr_id'].isin(res_stop['mr_id'].unique())]

Смотрим сколько маршрутов у нас есть и в валидациях, и в треках

In [37]:
len(validation4['mr_id'].unique())

7

In [38]:
validation4['tabelnum'].unique()

array([7366., 7502., 7386., 7378., 2958., 3847., 7356., 5958., 7700.,
       7388., 7714., 7374., 3851., 7710., 7696., 7364., 7380., 5960.,
       3855., 7390., 7708., 7368., 7370., 2738., 7382., 5956., 7354.,
       3861., 5957., 7718., 3870., 7441., 8718., 7360., 3843., 7575.,
       3884., 7704., 3886., 3890., 7706., 7698., 7724., 3849., 7523.,
       2952., 7720., 7394.])

In [39]:
res_stop2=res_stop

In [40]:
k=0
res=validation4
result = []
result2 = []
stops=pd.DataFrame()
for index, row in validation4.iterrows():
    stops=res_stop2[(res_stop2['mr_id']==row['mr_id'])
                    &(res_stop2['tabelnum']==row['tabelnum'])
                    &(res_stop2['time_start']<=row['time'])
                    &((res_stop2['time_finish']>row['time'])|(res_stop2['time_finish'].isna()))]
    if stops.empty:
        result.append(0)
        result2.append(0)
    else:
        stop_unic=stops['ssId'].values
        result.append(stop_unic[0])
        rl_id=stops['rl_id'].values
        result2.append(rl_id[0])
res['ssId']=result
res['rl_id']=result2
FOR_MATRIX=res

In [41]:
len(res[res['ssId']==0])

674

In [42]:
stops=pd.read_csv('./data/stop_seq_TB.csv', sep = ';', encoding='utf-8')

In [43]:
stops=stops[['ssId','ss_lat','ss_long']].drop_duplicates()

In [44]:
stops

,ssId,ss_lat,ss_long
0,stop__9854762,53.476261,49.349212
1,stop__9854369,53.478978,49.352483
2,stop__9854372,53.477996,49.358239
3,stop__9854374,53.478383,49.367120
4,stop__9854257,53.500550,49.389805
...,...,...,...
451,stop__9854655,53.482168,49.467528
452,stop__9854654,53.482987,49.460872
453,stop__9854652,53.477172,49.457782
454,stop__9854690,53.475421,49.462101


In [45]:
res2=res.merge(stops, on='ssId', how='left')

In [46]:
res2['c']=1

In [47]:
res2[res2['ssId']==0]['mr_id'].unique()

array([2227622944, 2227623020, 2227623040, 2227623078], dtype=int64)

In [48]:
rl_orderby=res_stop[['ssId','rl_id','mr_id','rc_orderby']].drop_duplicates()

In [49]:
rl_orderby.head()

,ssId,rl_id,mr_id,rc_orderby
0,stop__9854705,2.278607e+09,2.227623e+09,2.0
1,stop__9854703,2.278607e+09,2.227623e+09,3.0
2,stop__9854681,2.278607e+09,2.227623e+09,4.0
3,stop__9854335,2.278607e+09,2.227623e+09,5.0
4,stop__9854337,2.278607e+09,2.227623e+09,6.0


In [50]:
len(res2)

9575

In [51]:
res2=res2.merge(rl_orderby, on=['ssId','rl_id','mr_id'], how='left')

In [52]:
80874/288226

0.2805923129766225

In [53]:
len(res2[res2['ssId']==0])

674

## НЕ ЗАБЫТЬ ПОМЕНЯТЬ ИМЯ ФАЙЛА

In [54]:
res2.to_csv(path+'For_matrix_TB_2019-10-09.csv')

In [55]:
res2.head()

,Дата и время поездки,Терминал,Номер билета,Номер кристалла,TCAT,NUM_TRIP,Код Маршрута,Табельный номер,Код предприятия_x,Код предприятия_y,...,start,finish,tabelnum,tr_id,ssId,rl_id,ss_lat,ss_long,c,rc_orderby
0,2019-10-09 06:16:53,82784787,1,40AB3B79,БЕТК ТЛТ,0,42303456789,2500,26,26,...,22613,64228,7366.0,,stop__9854704,2.278607e+09,53.476982,49.473508,1,15.0
1,2019-10-09 06:16:56,82784787,2,8E756685,СОЦ,73,42303456789,2500,26,26,...,22613,64228,7366.0,,stop__9854704,2.278607e+09,53.476982,49.473508,1,15.0
2,2019-10-09 06:19:35,82784787,3,ADD3F69B,СОЦ,74,42303456789,2500,26,26,...,22613,64228,7366.0,,stop__9854704,2.278607e+09,53.476982,49.473508,1,15.0
3,2019-10-09 06:21:17,82784787,6,B30C5842,СОЦ,82,42303456789,2500,26,26,...,22613,64228,7366.0,,stop__9854704,2.278607e+09,53.476982,49.473508,1,15.0
4,2019-10-09 06:21:30,82784787,7,NaN,БАНК,0,42303456789,2500,26,26,...,22613,64228,7366.0,,stop__9854704,2.278607e+09,53.476982,49.473508,1,15.0


In [56]:
for_karta3=res2.groupby(['ssId','ss_lat','ss_long']).agg({'c': np.sum}).reset_index()
for_karta3=for_karta3.groupby(['ssId','ss_lat','ss_long']).agg({'c': np.sum}).reset_index()
gdf = gpd.GeoDataFrame(
    for_karta3, geometry=gpd.points_from_xy(x=for_karta3.ss_long, y=for_karta3.ss_lat)
)
gdf.crs='epsg:4326'
gdf.to_file(fr"./res/stops_2019_10_09.json", index=False, driver="GeoJSON")